In [1]:
import pandas as pd
!pip install plotly
!pip install dash
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [63]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [97]:

app = dash.Dash(__name__)
app.layout = html.Div(children = [
    html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40, 'font-family': 'Arial'}),
    dcc.Dropdown(id = 'site-dropdown', options = [
        {'label': 'All Sites', 'value': 'All'},
        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
    ],
                 value = 'All', placeholder = 'place holder here', searchable=True,
                 style= {'width': '80%', 'padding': '3px', 'text-size': '20px', 'textAlign': 'center', 'font-family': 'Arial'}
    ),
    html.Br(),
    html.Div(dcc.Graph(id = 'success-pie-chart')
    ),
    html.P("Payload range (Kg):", style = {'font-family': 'Arial'}),
    dcc.RangeSlider(id='payload-slider', min = min_payload, max = max_payload, step= 1000, value = [min_payload, max_payload]
    ),
    html.Div(dcc.Graph(id = 'success-payload-scatter-chart')
    )
])

@app.callback(
    Output(component_id = 'success-pie-chart', component_property='figure'),
    Input(component_id = 'site-dropdown', component_property = 'value')
)

def get_pie_chart(entered_site):
    if entered_site == 'All':
        data1 = spacex_df[spacex_df['class'] ==1].groupby('Launch Site').sum().reset_index()
        output =  px.pie(data1, values = 'class', names = 'Launch Site', title = 'Total Success Launches by Site')
        return output
    else:
        data2 = spacex_df[spacex_df['Launch Site'] == entered_site].reset_index()
        for i,e in enumerate(data2['class']):
            if e == 0:
                data2.loc[i,'total'] = data2['class'].value_counts()[0]
            else:
                data2.loc[i,'total'] = data2['class'].value_counts()[1]

        data2 = data2.groupby('class')['total'].mean().reset_index()
        output = px.pie(data2, values = 'total', names= 'class', title = 'Total Success Launches for Site {}'.format(entered_site))
        return output
    
@app.callback(
    Output(component_id = 'success-payload-scatter-chart', component_property='figure'),
    [Input(component_id = 'site-dropdown', component_property = 'value'),
     Input(component_id='payload-slider', component_property = 'value')]
)

def get_scatter_chart(input1, input2):
    new = spacex_df.copy()
    for i,e in enumerate(new['Payload Mass (kg)']):
        if e in range(input2[0], input2[1]):
            new.loc[i,'within'] = True
        else:
            new.loc[i,'within'] = False
    new2 = new[new['within'] == True].reset_index()
    if input1 == 'All':
        output = px.scatter(new2, x = 'Payload Mass (kg)', y = 'class', color="Booster Version Category",
                    title = 'correlation between Payload and success for All Sites', range_x = [-700,10500], range_y = [-0.1,1.2])
        return output
    else:
        data = new2[new2['Launch Site'] == input1]
        output = px.scatter(data, x = 'Payload Mass (kg)', y = 'class', color="Booster Version Category",
                    title = 'correlation between Payload and success for Site {}'.format(input1), range_x = [-700,10500], range_y = [-0.1,1.2])
        return output

if __name__ == '__main__':
    app.run_server(port = 8093)                           
            